This notebook demonstrates an orbit determination from position data using a Keplerian propagator and a Batch-Least-Squares (BLS) estimator.

It also includes an IOD (Initial Orbit Determination) to avoid needing to know a first guess of the orbit.

Parameters

In [1]:
sigma_position = 100e3  # Noise (in terms of standard deviation of gaussian distribution) of input position data in meters
sigma_velocity = 100.0  # Noise of input velocity data in meters per second

# Estimator parameters
estimator_position_scale = 1.0 # m
estimator_convergence_thres = 1e-2
estimator_max_iterations = 25
estimator_max_evaluations = 35

# Orbit propagator parameters
prop_min_step = 0.001 # s
prop_max_step = 300.0 # s
prop_position_error = 10.0 # m

Importing generated position/velocity data

In [2]:
import pandas as pd
points_df = pd.read_csv('pv_simulated_gcrf_with_noise.csv', index_col=0, parse_dates=True)
points_true_df = pd.read_csv('pv_simulated_gcrf_without_noise.csv', index_col=0, parse_dates=True)
display(points_df)

,x,y,z,vx,vy,vz
2020-01-01 01:15:00,-4.100845e+06,2.615425e+06,-4.759243e+06,-3623.980831,4010.938693,5317.404963
2020-01-01 01:23:20,-5.232941e+06,4.262905e+06,-1.471868e+06,-648.898649,1776.460829,7188.497350
2020-01-01 01:31:40,-4.787433e+06,4.498748e+06,2.118114e+06,2535.337108,-739.645375,7087.354134
2020-01-01 01:40:00,-3.082245e+06,3.354607e+06,5.293865e+06,4780.055703,-3277.485557,5013.488644
2020-01-01 01:48:20,-2.752019e+05,1.200146e+06,6.814466e+06,5754.039818,-4735.540685,1200.886810
2020-01-01 01:56:40,2.477261e+06,-1.099558e+06,6.374250e+06,5179.534246,-4861.776170,-3051.906538
2020-01-01 02:05:00,4.495547e+06,-3.211883e+06,3.978188e+06,2939.002015,-3409.018079,-6075.496180
2020-01-01 02:13:20,5.365940e+06,-3.997454e+06,3.923546e+05,-294.164942,-1238.876745,-7626.515473
2020-01-01 02:21:40,4.364125e+06,-4.251921e+06,-3.137996e+06,-3242.732550,1924.049850,-6786.544368
2020-01-01 02:30:00,2.208699e+06,-2.799695e+06,-5.970779e+06,-5369.700862,3606.795318,-4048.146862


Firing up a JVM for Orekit

In [3]:
import orekit
orekit.initVM()

Downloading and importing the Orekit data ZIP

In [4]:
from orekit.pyhelpers import download_orekit_data_curdir, setup_orekit_curdir
download_orekit_data_curdir()
setup_orekit_curdir()

In [5]:
from org.orekit.frames import FramesFactory
gcrf = FramesFactory.getGCRF()

from org.orekit.time import TimeScalesFactory
utc = TimeScalesFactory.getUTC()

Picking 3 vectors for orbit determination. These vectors are selected evenly-spaced in the dataframe.

In [6]:
import math
i = math.ceil(len(points_df.index) / 3)
points_for_iod = points_df.iloc[::i, :]
display(points_for_iod)

from org.hipparchus.geometry.euclidean.threed import Vector3D
from orekit.pyhelpers import datetime_to_absolutedate
pos_1 = points_for_iod.iloc[0]
vector_1 = Vector3D(pos_1[['x', 'y', 'z']].to_list())
date_1 = datetime_to_absolutedate(points_for_iod.index[0])

pos_2 = points_for_iod.iloc[1]
vector_2 = Vector3D(pos_2[['x', 'y', 'z']].to_list())
date_2 = datetime_to_absolutedate(points_for_iod.index[1])

pos_3 = points_for_iod.iloc[2]
vector_3 = Vector3D(pos_3[['x', 'y', 'z']].to_list())
date_3 = datetime_to_absolutedate(points_for_iod.index[2])

,x,y,z,vx,vy,vz
2020-01-01 01:15:00,-4.100845e+06,2.615425e+06,-4.759243e+06,-3623.980831,4010.938693,5317.404963
2020-01-01 01:48:20,-2.752019e+05,1.200146e+06,6.814466e+06,5754.039818,-4735.540685,1200.886810
2020-01-01 02:21:40,4.364125e+06,-4.251921e+06,-3.137996e+06,-3242.732550,1924.049850,-6786.544368


Performing the Initial Orbit Determination using Gibb's method. It assumes that at least 3 data points are available

In [7]:
from org.orekit.estimation.iod import IodGibbs
from org.orekit.utils import Constants as orekit_constants
iod_gibbs = IodGibbs(orekit_constants.EIGEN5C_EARTH_MU)
orbit_first_guess = iod_gibbs.estimate(gcrf,
                                      vector_1, date_1,
                                      vector_2, date_2,
                                      vector_3, date_3)
from org.orekit.propagation.analytical import KeplerianPropagator
kepler_propagator_iod = KeplerianPropagator(orbit_first_guess)

display(orbit_first_guess)

<KeplerianOrbit: Keplerian parameters: {a: 6864237.170672177; e: 0.009857787610219878; i: 96.22078964358812; pa: -71.93993560956903; raan: 140.0187077554949; v: 153.78728956760176;}>

Setting up a numerical propagator. It is not possible in Orekit to perform orbit determination with a Keplerian propagator.

In [8]:
from org.orekit.propagation.conversion import DormandPrince853IntegratorBuilder
integratorBuilder = DormandPrince853IntegratorBuilder(prop_min_step, prop_max_step, prop_position_error)

from org.orekit.propagation.conversion import NumericalPropagatorBuilder
from org.orekit.orbits import PositionAngle
propagatorBuilder = NumericalPropagatorBuilder(orbit_first_guess,
                                               integratorBuilder, PositionAngle.TRUE, estimator_position_scale)
from org.hipparchus.linear import QRDecomposer
matrix_decomposer = QRDecomposer(1e-11)
from org.hipparchus.optim.nonlinear.vector.leastsquares import GaussNewtonOptimizer
optimizer = GaussNewtonOptimizer(matrix_decomposer, False)

from org.orekit.estimation.leastsquares import BatchLSEstimator
estimator = BatchLSEstimator(optimizer, propagatorBuilder)
estimator.setParametersConvergenceThreshold(estimator_convergence_thres)
estimator.setMaxIterations(estimator_max_iterations)
estimator.setMaxEvaluations(estimator_max_evaluations)

Feeding position measurements to the esimator

In [9]:
from orekit.pyhelpers import datetime_to_absolutedate
from org.orekit.estimation.measurements import Position, ObservableSatellite

observableSatellite = ObservableSatellite(0) # Propagator index = 0

for timestamp, pv_gcrf in points_df.iterrows():
    orekit_position = Position(
        datetime_to_absolutedate(timestamp),
        Vector3D(pv_gcrf[['x', 'y', 'z']].to_list()),
        sigma_position,
        1.0,  # Base weight
        observableSatellite
    )
    estimator.addMeasurement(orekit_position)

Performing the orbit determination

In [10]:
estimatedPropagatorArray = estimator.estimate()

JavaError: <super: <class 'JavaError'>, <JavaError object>>
    Java stacktrace:
org.orekit.errors.OrekitException: Jacobian matrix for type KEPLERIAN is singular with current orbit
	at org.orekit.propagation.numerical.NumericalPropagator.tolerances(NumericalPropagator.java:684)
	at org.orekit.propagation.conversion.DormandPrince853IntegratorBuilder.buildIntegrator(DormandPrince853IntegratorBuilder.java:55)
	at org.orekit.propagation.conversion.NumericalPropagatorBuilder.buildPropagator(NumericalPropagatorBuilder.java:209)
	at org.orekit.propagation.conversion.NumericalPropagatorBuilder.buildPropagator(NumericalPropagatorBuilder.java:47)
	at org.orekit.estimation.leastsquares.BatchLSModel.createPropagators(BatchLSModel.java:323)
	at org.orekit.estimation.leastsquares.BatchLSModel.value(BatchLSModel.java:224)
	at org.hipparchus.optim.nonlinear.vector.leastsquares.LeastSquaresFactory$LocalLeastSquaresProblem.evaluate(LeastSquaresFactory.java:440)
	at org.orekit.estimation.leastsquares.Ba

In [ ]:
dt = 60.0
date_start = datetime_to_absolutedate(points_df.index[0])
date_end = datetime_to_absolutedate(points_df.index[-1])

# First propagating in ephemeris mode
estimatedPropagator = estimatedPropagatorArray[0]
estimatedInitialState = estimatedPropagator.getInitialState()
display(estimatedInitialState.getOrbit())
estimatedPropagator.resetInitialState(estimatedInitialState)
estimatedPropagator.setEphemerisMode()

estimatedPropagator.propagate(date_start, date_end)
bounded_propagator = estimatedPropagator.getGeneratedEphemeris()

Propagating the bounded propagator to retrieve the intermediate states

In [ ]:
# BLS = batch least squares

import numpy as np
from orekit.pyhelpers import absolutedate_to_datetime
pv_bls_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
pv_iod_df = pd.DataFrame(columns=['x', 'y', 'z', 'vx', 'vy', 'vz'])
    
date_current = date_start
while date_current.compareTo(date_end) <= 0:
    datetime_current = absolutedate_to_datetime(date_current)    
    spacecraftState = bounded_propagator.propagate(date_current)
    
    pv_bls = spacecraftState.getPVCoordinates(gcrf)
    pos_bls = np.array(pv_bls.getPosition().toArray())
    pv_bls_df.loc[datetime_current] = np.concatenate(
        (pos_bls,
         np.array(pv_bls.getVelocity().toArray()))
    )

    pv_iod = kepler_propagator_iod.getPVCoordinates(date_current, gcrf)
    pos_iod_gcrf = np.array(pv_iod.getPosition().toArray())
    pv_iod_df.loc[datetime_current] = np.concatenate(
        (pos_iod_gcrf,
         np.array(pv_iod.getVelocity().toArray()))
    )
    date_current = date_current.shiftedBy(dt)    

Plotting orbit in 3D.

In [ ]:
import plotly.io as pio
pio.renderers.default = 'jupyterlab+notebook+png'  # Uncomment for interactive plots

In [ ]:
import plotly.graph_objects as go
fig_data = data=[go.Scatter3d(x=pv_iod_df['x'], y=pv_iod_df['y'], z=pv_iod_df['z'],
                              mode='lines',
                              name='IOD solution'),
                 go.Scatter3d(x=pv_bls_df['x'], y=pv_bls_df['y'], z=pv_bls_df['z'],
                              mode='lines',
                              name='Batch least squares solution'),
                 go.Scatter3d(x=points_df['x'], y=points_df['y'], z=points_df['z'],
                             mode='markers',
                             name='Measurements'),
                 go.Scatter3d(x=points_for_iod['x'], y=points_for_iod['y'], z=points_for_iod['z'],
                             mode='markers',
                             name='Measurements used for IOD')]
scene=dict(aspectmode='data', #this string can be 'data', 'cube', 'auto', 'manual'
          )
layout = dict(
    scene=scene
)
fig = go.Figure(data=fig_data,
               layout=layout)
fig.show()

Computing residuals

In [ ]:
residuals_df = pd.DataFrame(columns=['bls_minus_measurements_norm', 'iod_minus_measurements_norm', 'bls_minus_truth_norm', 'iod_minus_truth_norm'])

for timestamp, pv_gcrf in points_df.iterrows():    
    date_current = datetime_to_absolutedate(timestamp)
    pv_bls = bounded_propagator.getPVCoordinates(date_current, gcrf)
    pos_bls = np.array(pv_bls.getPosition().toArray())

    pv_iod = kepler_propagator_iod.getPVCoordinates(date_current, gcrf)
    pos_iod = np.array(pv_iod.getPosition().toArray())
    
    pv_measurements = points_df.loc[timestamp]
    pos_measurements = pv_measurements[['x', 'y', 'z']]
    
    pv_true = points_true_df.loc[timestamp]
    pos_true = pv_true[['x', 'y', 'z']]
    
    bls_minus_measurements = np.linalg.norm(pos_bls - pos_measurements)
    iod_minus_measurements = np.linalg.norm(pos_iod - pos_measurements)
    bls_minus_truth = np.linalg.norm(pos_bls - pos_true)
    iod_minus_truth = np.linalg.norm(pos_iod - pos_true)
    
    residuals_df.loc[timestamp] = [
        np.linalg.norm(pos_bls - pos_measurements),
        np.linalg.norm(pos_iod - pos_measurements),
        np.linalg.norm(pos_bls - pos_true),
        np.linalg.norm(pos_iod - pos_true),
    ]
    
display(residuals_df)

Showing the residuals, i.e. the distance between the measurement points and the estimated positions (by the IOD and the BLS respectively).

This tells how well the estimation fits the measurements, but not how well it fits the "true" orbit, because the measurements contain significant noise here.

In [ ]:
fig = go.Figure(data=[
    go.Scatter(x=residuals_df.index, y=residuals_df['bls_minus_measurements_norm'], 
               name='BLS - measurements', 
               mode='markers+lines'),
    go.Scatter(x=residuals_df.index, y=residuals_df['iod_minus_measurements_norm'], 
               name='IOD - measurements', 
               mode='markers+lines')    
])

fig.show()

Showing the "estimation error", i.e. the difference between the truth (used to generate the measurement data) and the estimation.

The batch least squares estimation gives better results than the measurement noise: it is able to "filter out" the noise a little bit as it fits the ellipse.

In [ ]:
fig = go.Figure(data=[
    go.Scatter(x=residuals_df.index, y=residuals_df['bls_minus_truth_norm'], name='BLS - truth', mode='markers+lines'),
    go.Scatter(x=residuals_df.index, y=residuals_df['iod_minus_truth_norm'], name='IOD - truth', mode='markers+lines'),
])

fig.show()